# Cleaning Automation

In [1]:
import pandas as pd

In [2]:
def remove_tuples_with_less_data(data):
    """
    Remove tuples from the DataFrame where the number of non-null values is less than the specified minimum.

    Parameters:
    - data: pandas DataFrame

    Returns:
    - pandas DataFrame with tuples removed
    """
    # Display the number of columns in the DataFrame
    num_columns = len(data.columns)
    print(f"Number of columns in the DataFrame: {num_columns}")

    # Display the DataFrame to the user
    print("\nOriginal DataFrame:")
    print(data.head(10))

    # Ask the user for the minimum non-null count
    min_non_null_count = int(input("\nEnter the minimum number of non-null values required for each tuple: "))

    # Count non-null values for each tuple
    non_null_counts = data.count(axis=1)

    # Identify tuples with less than the specified minimum non-null values
    tuples_to_remove = data[non_null_counts < min_non_null_count]

    # Display information about tuples to be removed
    print(f"\nTuples to be removed:\n{tuples_to_remove}")

    # Ask for user confirmation
    user_input = input(f"\nDo you want to remove these tuples? (yes/no): ").lower()

    if user_input == 'yes':
        # Remove the identified tuples
        data_cleaned = data.drop(tuples_to_remove.index)
        print("Tuples removed successfully.")
        return data_cleaned
    else:
        print("No tuples were removed.")
        return data


# Example usage:
# Assuming 'your_data' is your DataFrame.

# your_data = ...

# Cleaned data after removing tuples with less data
# cleaned_data = remove_tuples_with_less_data(your_data)


In [3]:
def fill_nulls_with_3m(data):
    """
    Fill null values in specified columns based on user choice (mean/median/mode).

    Parameters:
    - data: pandas DataFrame

    Returns:
    - pandas DataFrame with null values filled
    """

    # Ask the user for a list of column names to fill
    columns_to_fill = input("\nEnter a comma-separated list of column names to fill (e.g., col1, col2): ").split(',')

    # Remove leading and trailing whitespaces from column names
    columns_to_fill = [col.strip() for col in columns_to_fill]

    # Iterate over selected columns and ask the user for the filling method
    for column in columns_to_fill:
        if column in data.columns:
            # Ask the user for the filling method (mean/median/mode)
            filling_method = input(f"\nChoose filling method for '{column}' (mean/median/mode): ").lower()

            # Fill null values based on user choice
            if filling_method == 'mean':
                data[column].fillna(data[column].mean(), inplace=True)
                print(f"Null values in '{column}' filled with mean.")
            elif filling_method == 'median':
                data[column].fillna(data[column].median(), inplace=True)
                print(f"Null values in '{column}' filled with median.")
            elif filling_method == 'mode':
                data[column].fillna(data[column].mode()[0], inplace=True)
                print(f"Null values in '{column}' filled with mode.")
            else:
                print(f"Invalid filling method '{filling_method}' for '{column}'. Skipping.")

    # Display the top 10 tuples of the DataFrame after filling null values
    print("\nDataFrame after filling null values (top 10 tuples):")
    print(data.head(10))

    return data


# Example usage:
# Assuming 'your_data' is your DataFrame.

# your_data = ...

# DataFrame after filling null values based on user choice
# data_filled = fill_nulls_with_user_choice(your_data)

In [4]:
def fill_nulls_with_global_constant(data):
    """
    Fill null values in columns with around 10% null values using a global constant.

    Parameters:
    - data: pandas DataFrame

    Returns:
    - pandas DataFrame with null values filled
    """

    # Identify columns with around 10% null values
    null_percentage_threshold = 10
    columns_with_nulls = data.columns[data.isnull().mean() * 100 > null_percentage_threshold]

    if not columns_with_nulls.empty:
        # Display columns with around 10% null values to the user
        print("\nColumns with around 10% null values:")
        print(columns_with_nulls)

        # Ask the user for a global constant value
        global_constant = input("\nEnter the global constant value to fill null values in these columns: ")

        # Iterate over selected columns and fill null values with the global constant
        for column in columns_with_nulls:
            data[column].fillna(global_constant, inplace=True)
            print(f"Null values in '{column}' filled with global constant.")

        # Display the top 10 tuples of the DataFrame after filling null values
        print("\nDataFrame after filling null values (top 10 tuples):")
        print(data.head(10))
    else:
        print("\nNo columns with around 10% null values found. No null values were filled.")

    return data


# Example usage:
# Assuming 'your_data' is your DataFrame.

# your_data = ...

# DataFrame after filling null values with a global constant for columns with around 10% null values
# data_filled = fill_nulls_with_global_constant(your_data)

In [5]:
def fill_nulls_by_class_user_input(data, class_column):
    """
    Fill null values in columns based on the user's choice of attribute mean or median, by class.

    Parameters:
    - data: pandas DataFrame
    - class_column: str, the column representing the class variable

    Returns:
    - pandas DataFrame with null values filled based on user input
    """

    # Check if the specified class column exists
    if class_column not in data.columns:
        print(f"\nError: '{class_column}' not found in the DataFrame.")
        return data

    # Identify columns with null values
    columns_with_nulls = data.columns[data.isnull().any()]

    # Create a table showing null values and their percentages by class
    null_summary_table = pd.DataFrame(index=columns_with_nulls, columns=['Null Count', 'Null Percentage by Class'])

    for column in columns_with_nulls:
        null_summary_table.loc[column, 'Null Count'] = data[column].isnull().sum()

        # Create a new DataFrame for the percentage by class operation
        percentage_by_class_df = pd.DataFrame(
            data.groupby(class_column)[column].apply(lambda x: x.isnull().mean() * 100)
        ).round(2).astype(str) + '%'

        # Assign the values to the null_summary_table
        null_summary_table.loc[column, 'Null Percentage by Class'] = percentage_by_class_df.values

    # Display the null summary table
    print("\nNull Values Summary Table:")
    print(null_summary_table)

    # Ask the user for the list of columns to fill and their filling method (mean/median)
    fill_columns = input("\nEnter a comma-separated list of columns to fill (e.g., col1, col2): ").split(',')
    fill_method = input("\nEnter filling method for these columns (mean/median): ").lower()

    # Remove leading and trailing whitespaces from column names
    fill_columns = [col.strip() for col in fill_columns]

    # Iterate over selected columns and fill null values based on user input
    for column in fill_columns:
        if column in data.columns:
            if fill_method == 'mean':
                data[column] = data.groupby(class_column)[column].transform(lambda x: x.fillna(x.mean()))
            elif fill_method == 'median':
                data[column] = data.groupby(class_column)[column].transform(lambda x: x.fillna(x.median()))
            else:
                print(f"Invalid filling method '{fill_method}' for '{column}'. Skipping.")

            print(f"Null values in '{column}' filled based on {fill_method} by class.")

    # Display the top 10 tuples of the DataFrame after filling null values
    print("\nDataFrame after filling null values (top 10 tuples):")
    print(data.head(10))

    return data

# Example usage:
# Assuming 'data' is your DataFrame and 'class_column' is the column representing the class variable
# fill_nulls_by_class_user_input(data, 'class_column')


In [6]:
import pandas as pd

def remove_columns_with_high_null_percentage(data):
    """
    Remove columns with null values exceeding the specified threshold percentage.

    Parameters:
    - data: pandas DataFrame

    Returns:
    - pandas DataFrame with columns removed
    """
    # Calculate the percentage of null values for each column
    null_percentage = (data.isnull().mean() * 100).round(2)
    print(null_percentage)
    # Specify the threshold percentage
    threshold_percentage = float(input("Enter threshold percentage: "))

    # Identify columns above the threshold
    columns_above_threshold = null_percentage[null_percentage < threshold_percentage].index.tolist()

    if columns_above_threshold:
        # Display information about columns to be removed
        print(f"\nColumns to be removed (null percentage > {threshold_percentage}%):\n{columns_above_threshold}")

        # Ask for user confirmation
        user_input = input("\nDo you want to remove these columns? (yes/no): ").lower()

        if user_input == 'yes':
            # Remove the identified columns
            data_cleaned = data.drop(columns=columns_above_threshold)
            print("Columns removed successfully.")
            return data_cleaned
        else:
            print("No columns were removed.")
            return data
    else:
        print("\nNo columns with null values exceeding the threshold found. No columns were removed.")
        return data

# Example usage:
# Assuming 'your_data' is your DataFrame.
# your_data = ...

# Cleaned data after removing columns with high null percentage
# cleaned_data = remove_columns_with_high_null_percentage(your_data)


In [7]:

def main():
    # Step 1: Take data and convert it into a DataFrame
    file_path = input("Enter the path to the CSV file: ")
    
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
        return
    except pd.errors.EmptyDataError:
        print(f"Error: The file '{file_path}' is empty.")
        return

    # Display the original DataFrame

    # Step 2: Remove columns with high null percentage
    df = remove_columns_with_high_null_percentage(df)  # Adjust the threshold percentage as needed

    # Display the DataFrame after removing columns with high null percentage
    print("\nDataFrame after removing columns with high null percentage:")
    print(df)
    # Step 2.1: Remove tuples with less data
    df = remove_tuples_with_less_data(df)  # Adjust the minimum non-null count as needed

    # Display the DataFrame after removing tuples with less data
    print("\nDataFrame after removing tuples with less data:")
    print(df)

    

    # Initialize a loop until there are no null values
    while df.isnull().any().any():
        # Display the columns with their corresponding percentage of null values
        print("\nColumns and their percentage of null values:")
        null_percentage = df.isnull().mean() * 100
        print(null_percentage)

        print("\nNull values are present in the DataFrame.")

        # Display menu for the user to choose a method
        print("\nChoose a method to handle null values:")
        print("1. Fill with mean/median/mode")
        print("2. Fill with global constant")
        print("3. Fill by class with user input")

        choice = input("Enter your choice (1/2/3): ")

        if choice == '1':
            # Step 3: Fill null values with user choice
            df = fill_nulls_with_3m(df)

        elif choice == '2':
            # Step 4: Fill null values with a global constant for columns with around 10% null values
            df = fill_nulls_with_global_constant(df)

        elif choice == '3':
            # Step 5: Fill null values by class with user input
            class_column = input("Enter the column representing the class variable: ")
            df = fill_nulls_by_class_user_input(df, class_column)

        else:
            print("Invalid choice. Please enter 1, 2, or 3.")

    # Display the final DataFrame without null values
    print("\nFinal DataFrame without null values:")
    print(df)

if __name__ == "__main__":
    main()


Enter the path to the CSV file: demographic.csv
SEQN         0.00
SDDSRVYR     0.00
RIDSTATR     0.00
RIAGENDR     0.00
RIDAGEYR     0.00
RIDAGEMN    93.39
RIDRETH1     0.00
RIDRETH3     0.00
RIDEXMON     3.56
RIDEXAGM    58.59
DMQMILIZ    38.47
DMQADFC     94.66
DMDBORN4     0.00
DMDCITZN     0.04
DMDYRSUS    81.25
DMDEDUC3    72.45
DMDEDUC2    43.30
DMDMARTL    43.30
RIDEXPRG    87.14
SIALANG      0.00
SIAPROXY     0.01
SIAINTRP     0.00
FIALANG      1.19
FIAPROXY     1.19
FIAINTRP     1.19
MIALANG     28.15
MIAPROXY    28.14
MIAINTRP    28.13
AIALANGA    37.92
DMDHHSIZ     0.00
DMDFMSIZ     0.00
DMDHHSZA     0.00
DMDHHSZB     0.00
DMDHHSZE     0.00
DMDHRGND     0.00
DMDHRAGE     0.00
DMDHRBR4     2.92
DMDHREDU     2.89
DMDHRMAR     1.21
DMDHSEDU    47.50
WTINT2YR     0.00
WTMEC2YR     0.00
SDMVPSU      0.00
SDMVSTRA     0.00
INDHHIN2     1.31
INDFMIN2     1.21
INDFMPIR     7.71
dtype: float64
Enter threshold percentage: 40

Columns to be removed (null percentage > 40.0%):
['RIDAGEMN

Enter your choice (1/2/3): 2

Columns with around 10% null values:
Index(['DMQMILIZ', 'MIALANG', 'MIAPROXY', 'MIAINTRP', 'AIALANGA'], dtype='object')

Enter the global constant value to fill null values in these columns: 25
Null values in 'DMQMILIZ' filled with global constant.
Null values in 'MIALANG' filled with global constant.
Null values in 'MIAPROXY' filled with global constant.
Null values in 'MIAINTRP' filled with global constant.
Null values in 'AIALANGA' filled with global constant.

DataFrame after filling null values (top 10 tuples):
    SEQN  SDDSRVYR  RIDSTATR  RIAGENDR  RIDAGEYR  RIDRETH1  RIDRETH3  \
0  73557         8         2         1        69         4         4   
1  73558         8         2         1        54         3         3   
2  73559         8         2         1        72         3         3   
3  73560         8         2         1         9         3         3   
4  73561         8         2         2        73         3         3   
5  73562        

Enter your choice (1/2/3): 1

Enter a comma-separated list of column names to fill (e.g., col1, col2): RIDEXMON,INDHHIN2,INDFMIN2,DMDHRBR4

Choose filling method for 'RIDEXMON' (mean/median/mode): mean
Null values in 'RIDEXMON' filled with mean.

Choose filling method for 'INDHHIN2' (mean/median/mode): median
Null values in 'INDHHIN2' filled with median.

Choose filling method for 'INDFMIN2' (mean/median/mode): mode
Null values in 'INDFMIN2' filled with mode.

Choose filling method for 'DMDHRBR4' (mean/median/mode): mean
Null values in 'DMDHRBR4' filled with mean.

DataFrame after filling null values (top 10 tuples):
    SEQN  SDDSRVYR  RIDSTATR  RIAGENDR  RIDAGEYR  RIDRETH1  RIDRETH3  \
0  73557         8         2         1        69         4         4   
1  73558         8         2         1        54         3         3   
2  73559         8         2         1        72         3         3   
3  73560         8         2         1         9         3         3   
4  73561       

Enter your choice (1/2/3): 3
Enter the column representing the class variable: DMDHRBR4

Null Values Summary Table:
         Null Count              Null Percentage by Class
SIAPROXY          1     [[0.0%], [0.0%], [0.03%], [0.0%]]
FIALANG         121  [[0.86%], [12.12%], [0.87%], [0.0%]]
FIAPROXY        121  [[0.86%], [12.12%], [0.87%], [0.0%]]
FIAINTRP        121  [[0.86%], [12.12%], [0.87%], [0.0%]]
DMDHREDU        294    [[0.0%], [98.99%], [0.0%], [0.0%]]
DMDHRMAR        123   [[1.26%], [4.38%], [0.77%], [0.0%]]

Enter a comma-separated list of columns to fill (e.g., col1, col2): DMDHRMAR

Enter filling method for these columns (mean/median): median
Null values in 'DMDHRMAR' filled based on median by class.

DataFrame after filling null values (top 10 tuples):
    SEQN  SDDSRVYR  RIDSTATR  RIAGENDR  RIDAGEYR  RIDRETH1  RIDRETH3  \
0  73557         8         2         1        69         4         4   
1  73558         8         2         1        54         3         3   
2  73559 

Enter your choice (1/2/3): 1

Enter a comma-separated list of column names to fill (e.g., col1, col2): 1

DataFrame after filling null values (top 10 tuples):
    SEQN  SDDSRVYR  RIDSTATR  RIAGENDR  RIDAGEYR  RIDRETH1  RIDRETH3  \
0  73557         8         2         1        69         4         4   
1  73558         8         2         1        54         3         3   
2  73559         8         2         1        72         3         3   
3  73560         8         2         1         9         3         3   
4  73561         8         2         2        73         3         3   
5  73562         8         2         1        56         1         1   
6  73563         8         2         1         0         3         3   
7  73564         8         2         2        61         3         3   
8  73565         8         1         1        42         2         2   
9  73566         8         2         2        56         3         3   

   RIDEXMON DMQMILIZ  DMDBORN4  ...  DMDHRBR4  D

Enter your choice (1/2/3): 1

Enter a comma-separated list of column names to fill (e.g., col1, col2): SIAPROXY

Choose filling method for 'SIAPROXY' (mean/median/mode): mean
Null values in 'SIAPROXY' filled with mean.

DataFrame after filling null values (top 10 tuples):
    SEQN  SDDSRVYR  RIDSTATR  RIAGENDR  RIDAGEYR  RIDRETH1  RIDRETH3  \
0  73557         8         2         1        69         4         4   
1  73558         8         2         1        54         3         3   
2  73559         8         2         1        72         3         3   
3  73560         8         2         1         9         3         3   
4  73561         8         2         2        73         3         3   
5  73562         8         2         1        56         1         1   
6  73563         8         2         1         0         3         3   
7  73564         8         2         2        61         3         3   
8  73565         8         1         1        42         2         2   
9  7356